# Initial Sizing

This section describes initial weight estimation method and demonstrates it on the example aircraft. The maximum takeoff weight ($W_{TO}$) can be divied into four components for this initial estimation: crew ($W_c$), payload ($W_p$), empty weight ($W_e$), and fuel weight ($W_f$). Mathematically, 
it can be written as

$$
    W_{TO} = W_c + W_p + W_e + W_f.
$$

Note that $W_p$ includes passenger and baggage both. The $W_c$ and $W_p$ are usually provided in the RFP. Following subsections outlines methods for 
computing $W_e$, $W_f$ and $W_{TO}$.

Below code snippet imports required python packages:

In [2]:
import numpy as np

## Empty Weight Fraction

During the intial phase of design, the empty weight is usually computed based on the historical data for similar type of aircraft.
Most aircraft design books will provide some sort of empirical formula to compute $W_e$ or empty weight fraction ($W_e/W_{TO}$). For the example airplane, empty weight trend provided in Nicolai and Carichner's book for light propeller airplane (Pg. 763) is used. The empty weight fraction is computed using

$$
    \frac{W_e}{W_{TO}} = 0.911 \times W_{TO}^{0.947}.
$$

> **_NOTE:_** In the above formula, $W_{TO}$ should be in pounds (lbs)

Following code defines a simple python function for computing $W_e/W_{TO}$ which is later used for computing $W_{TO}$.

In [3]:
def compute_empty_weight_fraction(wto):
    
    # empty weight fraction (from Nicolai and Carichner, Pg. 763)
    we_by_wto = 0.911 * wto ** 0.947
    
    return we_by_wto

## Fuel Fraction

The fuel fraction is computed using the mission segment weight fraction method. In this method, weight fractions are
computed for each segment of the mission profile and are multiplied to obtain the ratio of final weight ($W_{final}$) to initial weight i.e. maximum takeoff weight ($W_{TO}$). Mathematically, it can be written as

$$
    \frac{W_{final}}{W_{TO}} = \prod_{i=1}^{n} \frac{W_{i}}{W_{i-1}},
$$

where $W_{i-1}$ and $W_{i}$ are the weights at the start and end of mission segment $i$, respectively, and $n$ is the total number of mission segments. Note that $W_0$ denotes maximum takeoff weight ($W_{TO}$). The weight fractions can be computed using either historical data or simple analytical formulas, most aircraft design books will provide these details. Once final weight fraction is obtained, one can compute fuel fraction using

$$
    \frac{W_f}{W_{TO}} = 1.06 * \Big ( 1 - \frac{W_{final}}{W_{TO}} \Big) 
$$

Note that a $6\%$ factor is multiplied to account for trapped and unusable fuel based on the recommendations provided in Raymer's book.

This method is now used for computing fuel weight fraction for the example airplane. Based on the [mission profile](example_aircraft.md), there are 8 mission segments in total. The value of weight fractions for most mission segments is based on historical data, except for cruise and loiter mission. The historical data is obtained from Roskam Part 1 Table 2.2 (Pg 12) and is outlined in following table.

<div style="width:60%; margin: auto;">

Mission segment | Weight fraction | Value
------ | :-------: | :-------:
Warmup, taxi, takeoff | $W_1/W_0$ | 0.984
Climb | $W_2/W_1$ | 0.990
Descent | $W_4/W_3$ | 0.992
Climb | $W_5/W_4$ | 0.990
Descent | $W_7/W_6$ | 0.992
Land, taxi, shutdown | $W_8/W_7$ | 0.992
</div>

The cruise and loiter weight fractions are computed using following equations obtained from Breguet range and loiter equations, refer to lecture notes or any of the aircraft design books for more details.

$$
\begin{aligned}
    \text{cruise weight fraction:} \quad & \frac{W_3}{W_2} = \exp \Bigg[ - \frac{ RC_{{bhp}_{cr}} }{ 550\eta_{p_{cr}} (L/D)_{cr} } \Bigg] \\
    \text{loiter weight fraction:} \quad & \frac{W_6}{W_5} = \exp \Bigg[ - \frac{ E V_{lo} C_{{bhp}_{lo}} }{ 550 \eta_{p_{lo}} (L/D)_{lo} } \Bigg]
\end{aligned}
$$

> **_NOTE:_** The cruise and loiter equations are valid for propeller-driven airplanes only. All the units should be in FPS system i.e range should be in ft, loiter velocity should be in ft/s and E should be in hr.

The R and E are the range and endurance time, respectively. The C_bhp denotes brake specific fuel consumption for the piston-engine which is the pounds of fuel per hour to produce one horsepower at propeller shaft. The factor 550 converts lbs/(hr-hp) to lbs/(hr-lbs). The $\eta$ denotes propeller efficiency while $V$ is the speed of the airplane in ft/s. The subscript $cr$ and $lo$ denotes cruise and loiter, respectively. For a propeller-driven airplane, the cruise and loiter $L/D$ can be approximated as $L/D_{max}$ and $0.866 \times L/D_{max}$, respectively. The maximum $L/D$ for a subsonic airplane can be computed using 

$$
    \bigg(\frac{L}{D}\bigg)_{max} = \frac{1}{2\sqrt{C_{D_0}K}},
$$

where $C_{D_0}$ is the zero-lift drag coefficient and $K$ is induced drag factor given by 

$$
    K = \frac{1}{\pi Ae}.
$$

The $A$ is the aspect ratio and $e$ is oswald efficiency factor which is computed based on a statistical estimation provided in Raymer. The value of $A$ is assumed based on similar type of aircraft at this stage of design. The value of $C_{D_0}$ can either be estimated using historical data, but this example follows a more rigorous process based on $W_{TO}$ and wetted area ($S_{wet}$), outlined in Roskam Part 1, Section 3.4.1. Below table summarizes various fixed values used in fuel fraction computation.

<div style="width:100%; margin: auto;">

Parameter | Value | Source
-----| :----: | :-----:
Aspect ratio ($A$) | 8 | Assumed using similar aircraft
Cruise propeller efficiency ($\eta_{p_{cr}}$) | 0.82 | Roskam Part 1, Table 2.2
Cruise brake specific fuel consumption ($C_{{bhp}_{cr}}$) | 0.4 lb/hr/bhp | Raymer, Table 3.4
Loiter propeller efficiency ($\eta_{p_{lo}}$) | 0.72 | Roskam Part 1, Table 2.2
Loiter brake specific fuel consumption ($C_{{bhp}_{lo}}$) | 0.4 lb/hr/bhp | Raymer, Table 3.4
Loiter velocity ($V_{lo}$) | 120 KTAS | Assumed
</div>

Below code block defines functions for computing fuel fraction based on the $W_{TO}$.

In [4]:
# def compute_zero_lift_drag_coeff(wto):
#     """
#         Function for computing zero-lift drag coefficient,
#         refer to Roskam Part 1 Section 3.4.1
#     """

#     a =
#     b = 
#     c = 
#     d = 

#     f = 10**(a + b*())

#     pass f/S

compute_oswald_efficiency = lambda aspect_ratio: 1.78*(1 - 0.045*aspect_ratio**0.68) - 0.64 # Raymer, equation 12.48

def compute_fuel_fraction(wto):

    ######### define and compute variables

    # Range
    range = 1200 # nmi
    range = range * 1852 / 0.3048 # ft

    # Endurance
    E = 45/60 # in hour

    # Loiter speed
    V_lo = 120 # KTAS, assumed
    V_lo = V_lo * 1852 / 3600 / 0.3048 # in ft/s

    aspect_ratio = 8 # assumed

    # Propeller efficieny
    np_cr = 0.82
    np_lo = 0.72

    # C_bhp
    C_bhp_cr = 0.4
    C_bhp_lo = 0.4

    e = 1.78*(1 - 0.045*aspect_ratio**0.68) - 0.64 # oswald efficiency

    K = 1 / np.pi / aspect_ratio / e # induced drag factor

    zero_lift_drag_coeff = 0.0026

    L_by_D_max = 1 / 2 / (zero_lift_drag_coeff*K)**0.5 # maximum L_by_D

    L_by_D_cr = L_by_D_max

    L_by_D_lo = 0.866*L_by_D_max

    ######### Mission segment weight fractions

    w1_by_wto = 0.984 # warmup, taxi, takeoff

    w2_by_w1 = 0.99 # climb

    w3_by_w2 = np.exp(-range*C_bhp_cr/550/np_cr/L_by_D_cr) # cruise

    w4_by_w3 = 0.992 # descent

    w5_by_w4 = 0.99 # climb

    w6_by_w5 = np.exp(-E*V_lo*C_bhp_lo/550/np_lo/L_by_D_lo) # loiter

    w7_by_w6 = 0.992 # descent

    w8_by_w7 = 0.992 # land, taxi, shutdown

    w8_by_wto = w1_by_wto * w2_by_w1 * w3_by_w2 * w4_by_w3 \
                * w5_by_w4 * w6_by_w5 * w7_by_w6 * w8_by_w7

    wf_by_wto = 1.06 * (1 - w8_by_wto) # 6% is for trapped unusable fuel

    return wf_by_wto

## MTOW Estimation

$$
    W_{TO} = \frac{W_p}{1 - \dfrac{W_e}{W_{TO}} - \dfrac{W_f}{W_{TO}}}
$$

In [7]:
wto_guess = 5000
error = np.inf

wc = 200 # lbs
wp = 1000 # lbs

while error > 0.01:

    wto = (wc + wp) / (1 - compute_empty_weight_fraction(wto_guess) - compute_fuel_fraction(wto_guess))

    error = np.abs(wto - wto_guess)

    wto_guess = wto

/tmp/ipykernel_6163/5526747.py:4: RuntimeWarning: invalid value encountered in scalar power
  we_by_wto = 0.911 * wto ** 0.947
